In [1]:
# N = 10
# M = 15
# timeperiod = 3
# 每三天将TOP15外的轮出，轮入TOP10的

import requests
import pandas as pd
import json
'''
按照三低增强型轮动
选取10只
轮入：进入top10
轮出：掉出top15
设置当前total_value
等市值计算每只应该买入多少
每三天轮动一次，由于轮动周期短，单只股票不设置止损线了
'''
HEADERS = {
    "Accept": "application/json, text/plain, */*",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept-Language": "zh-CN,zh;q=0.9,en;q=0.8,en-GB;q=0.7,en-US;q=0.6,zh-TW;q=0.5,fr;q=0.4,ja;q=0.3",
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36 Edg/100.0.1185.36"
    "AppleWebKit/537.36 (KHTML, like Gecko) "
    "Chrome/54.0.2840.100 Safari/537.36",
    # "X-Requested-With": "XMLHttpRequest",
    "Cache-Control": "no-cache",
    "Init": "1",
    "DNT": "1"
    # "Columns": "1,70,2,3,5,6,11,12,14,15,16,29,30,32,34,35,75,44,46,47,52,53,54,56,57,58,59,60,62,63,67",
    # "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8"
}
# COOKIE = 'none'
session = requests.Session()
session.headers.update(HEADERS)

def login():
    url = "https://www.jisilu.cn/webapi/account/login_process/"
    session.post(url,{
        'return_url': 'https://www.jisilu.cn/',
        'user_name': 'd602b36bb6ea1b105b6b2c7c553d5bb6',
        'password': 'ed1d6fa706437d266d092d2c7ba1c074',
        'aes': '1'
    })



def get_bonds():
    headers = ['bond_id', 'bond_nm', 'price','premium_rt', 'curr_iss_amt', 'increase_rt','days_left']
    df = pd.DataFrame(data=None, columns=headers)
    # session.headers.update({'cookie': COOKIE})
    url = 'https://www.jisilu.cn/webapi/cb/list/'
    data = session.post(url)
    json_data = json.loads(data.text)
    if json_data['code'] == 200:
        bonds = json_data['data']
    # print('共有', len(bonds))
    print(JSON.stringify(bonds))
    for bond in bonds:
        if 'C' == bond['btype'] and bond['list_dt'] is not None: # 是可转债 & 已上市
            if 'R' == bond['redeem_icon'] or 'O' == bond['redeem_icon'] or bond['year_left'] > 5.99 or bond['increase_rt'] < -5 or bond['price'] == 100.00:  # 过滤公告强赎的
                continue
            interest = []
            for key in headers:
                if key in bond:
                    value = bond[key]
                    if key == 'bond_nm' and 'B' == bond['redeem_icon']:  # 已满足强赎的提示
                        value += '!'
                    interest.append(value)
            interest.append(int(365 * float(bond['year_left'])))
            # print(interest)
            df.loc[len(df)] = interest
    # print('上市非强赎共有', len(df))
    return df

print('start')
login()
print('logined')


# triple_low_enhance = df.sort_values(by=['factor'])



start
logined


In [1]:
import requests
import pandas as pd
import json
import re
from lxml import etree

URL = 'https://www.ninwin.cn/index.php?m=cb&show_cb_only=Y&show_listed_only=Y'
HEADERS = {
    "Accept": "application/json, text/plain, */*",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept-Language": "zh-CN,zh;q=0.9,en;q=0.8,en-GB;q=0.7,en-US;q=0.6,zh-TW;q=0.5,fr;q=0.4,ja;q=0.3",
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36 Edg/100.0.1185.36"
    "AppleWebKit/537.36 (KHTML, like Gecko) "
    "Chrome/54.0.2840.100 Safari/537.36",
    # "X-Requested-With": "XMLHttpRequest",
    "Cache-Control": "no-cache",
    "Init": "1",
    "DNT": "1"
    # "Columns": "1,70,2,3,5,6,11,12,14,15,16,29,30,32,34,35,75,44,46,47,52,53,54,56,57,58,59,60,62,63,67",
    # "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8"
}

session2 = requests.Session()
session2.headers.update(HEADERS)

data = session2.get(URL)
html = etree.HTML(data.text)
withdraw_list = []
withdraw_list_code = []
bonds = html.xpath('//*[@id="cb_hq"]/tbody/tr[*]')
pattern = re.compile(u'最快[1-9]\d*个交易日后可能满足强赎条件')
for bond in bonds:
    bond_id = bond.attrib.get('data-cbcode')
    bond_remain = -1
    bond_span = bond.find('td[3]/a/span')
    if bond_span != None:
        bond_str = bond_span.attrib.get('title')
        if pattern.search(bond_str):
            bond_remain = int(bond_str[3])
    if bond_remain > 0 and bond_remain <= 3:
        withdraw_list.append({
            'bond_id':bond_id,
            'withdraw_days': bond_remain
        })
        withdraw_list_code.append(bond_id)
    # print(bond_id, ' -> ', bond_remain)
print(withdraw_list)

[{'bond_id': '123156', 'withdraw_days': 4}, {'bond_id': '127069', 'withdraw_days': 4}, {'bond_id': '123118', 'withdraw_days': 4}, {'bond_id': '127014', 'withdraw_days': 2}]


In [2]:
df = get_bonds()
print('data loaded')
# df
df['premium_rt'] = df['premium_rt'].astype('float')
df['factor'] = 8 * df['premium_rt'] + df['price'] + df['curr_iss_amt']

triple_low_enhance = df.sort_values(by=['factor']).head(25)

# 在这里填入当前最新净值
total_value = 300000
triple_low_enhance['amount'] = triple_low_enhance.apply(
    lambda x: round(int(((total_value/15 + 800)/float(x.price))/10)*10), axis=1)
triple_low_enhance[['bond_id', 'bond_nm', 'price', 'factor', 'amount', 'premium_rt', 'curr_iss_amt', 'increase_rt']]

共有 499
[{'bond_id': '123181', 'bond_nm': '亚康转债', 'bond_py': 'ykzz', 'price': 100, 'increase_rt': 0, 'stock_id': '301085', 'stock_nm': '亚康股份', 'stock_py': 'ykgf', 'sprice': 40.69, 'sincrease_rt': 1.6, 'pb': 4.22, 'convert_price': 38.25, 'convert_value': 106.38, 'convert_dt': 184, 'premium_rt': -6, 'dblow': 94, 'sw_cd': '710301', 'market_cd': 'szcy', 'btype': 'C', 'list_dt': None, 'qflag2': 'N', 'owned': 0, 'hold': 0, 'bond_value': None, 'rating_cd': 'A+', 'option_value': None, 'put_convert_price': 26.78, 'force_redeem_price': 49.73, 'convert_amt_ratio': 20.2, 'fund_rt': None, 'short_maturity_dt': '29-03-20', 'year_left': 5.986, 'curr_iss_amt': 2.61, 'volume': 0, 'svolume': 19088.71, 'turnover_rt': 0, 'ytm_rt': 3.35, 'put_ytm_rt': None, 'noted': 0, 'bond_nm_tip': '', 'redeem_icon': '', 'last_time': None, 'qstatus': '00', 'margin_flg': 'R', 'sqflag': 'Y', 'pb_flag': 'N', 'adj_cnt': 0, 'adj_scnt': 0, 'convert_price_valid': 'Y', 'convert_price_tips': '', 'convert_cd_tip': '未到转股期；2023-09-27 

,bond_id,bond_nm,price,premium_rt,curr_iss_amt,increase_rt,days_left
0,123118,惠城转债,231.650,-0.63,2.462,-0.06,1561
1,128114,正邦转债,93.580,-0.07,13.117,0.51,1177
2,113595,花王转债,127.763,0.59,3.076,-3.82,1211
3,110061,川投转债,150.493,2.34,34.820,-1.24,958
4,127058,科伦转债,164.565,2.62,20.949,-0.26,1818
...,...,...,...,...,...,...,...
467,123117,健帆转债,109.401,215.07,9.998,-0.04,1547
468,123072,乐歌转债,131.700,224.06,1.419,0.15,1303
469,113584,家悦转债,104.394,224.60,6.449,0.04,1165
470,123013,横河转债,509.445,246.59,0.379,-0.11,486


In [2]:

df = get_bonds()
print('data loaded')
# df
df['premium_rt'] = df['premium_rt'].astype('float')
df['factor'] = 8 * df['premium_rt'] + df['price'] + df['curr_iss_amt']

triple_low_enhance = df.sort_values(by=['factor']).head(20)

# 在这里填入当前最新净值
total_value = 300000
triple_low_enhance['amount'] = triple_low_enhance.apply(
    lambda x: round(int(((total_value/15 + 800)/float(x.price))/10)*10), axis=1)
triple_low_enhance[['bond_id', 'bond_nm', 'price', 'factor', 'amount', 'premium_rt', 'curr_iss_amt', 'increase_rt']]

共有 488
上市非强赎共有 471
data loaded


,bond_id,bond_nm,price,factor,amount,premium_rt,curr_iss_amt,increase_rt
1,113570,百达转债,139.835,140.3120,140,-0.27,2.574,-2.38
11,128087,孚日转债,129.525,142.4075,160,3.22,6.445,0.73
5,128081,海亮转债,130.041,146.3830,150,0.72,28.364,0.11
0,113622,杭叉转债,141.975,146.7475,140,-0.29,11.285,-2.40
13,111004,明新转债,133.416,147.3105,150,3.51,6.729,1.08
96,128114,正邦转债,96.874,148.8490,210,15.13,13.170,-0.23
68,128014,永东转债,112.298,149.0525,180,11.69,3.369,-0.05
16,113647,禾丰转债,128.054,149.6990,160,4.77,14.670,0.89
12,111002,特纸转债,138.500,152.0110,150,3.40,6.622,-0.15
33,110063,鹰19转债,122.488,153.4040,160,7.29,18.092,-0.04
